In [1]:
'''
All rights can reffer to the LICENSE.md.

Created on July 19, 2018.

This module provides the several classes of autoencoder series.
The use could simply use these API without defining the structure by oneself.

@author: steven.cy.chuang
'''

'\nAll rights can reffer to the LICENSE.md.\n\nCreated on July 19, 2018.\n\nThis module provides the several classes of autoencoder series.\nThe use could simply use these API without defining the structure by oneself.\n\n@author: steven.cy.chuang\n'

In [2]:
import os
import pickle
import json
from time import time
import keras
from keras.layers import *
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model, model_from_json
from keras import backend as K
from keras.metrics import *

Using TensorFlow backend.


# Variational autoencoder

In [6]:
class VAE():
    _stdEps = 1.0
    
    def __init__(self, 
                 dimInput, 
                 layerDense=[64, 2], actDense='leaky_relu', dropoutDense=0.5,
                 batchNorm=True,
                 ratRecon=0.998
                ):
        """
        The basic properties and pipeline will be defined in the initialization.
        It should be noted that layerDense defines the first half(encoder) of network. 
        The decoder will be reflected structure.
        For example, [64, 16, 2] means the nodes of decoder will be [2, 16, 64]. 
        There is another parameter should noted that ratRecon=0.5 doesn't mean the effect is half.
        Because KL loss and reconstruction loss are not the same scale.
        Args:
            dimInput (int): the number of input dimension. All features are flatten as a vector.
            layerDense (list[int]): the numbers of each dense layer. Default is [64, 2].
            actDense (string): the activation function. Default is 'leaky_relu'.
            dropoutDense (float): the dropout layer. Default is 0.5.
            ratRecon (float): the parameter for tuning the effects between KL loss and reconstruction loss.
        """
        
        # Initialize some setting 
        self._dimInput = dimInput # all features are flatten as a vector
        self._inputs = Input(shape=(dimInput,)) 
        self._dimEncode = layerDense[-1]
        self._ratRecon = ratRecon
        
        self._encoding(layerDense, batchNorm, actDense, dropoutDense)
        
        self._decoding(layerDense, batchNorm, actDense, dropoutDense)
        
        self.autoencoder = Model(self._inputs, self.decoder(self.encoder(self._inputs)), name='autoencoder')

        
    def _stackDense(self, x, layerDense, batchNorm, actDense, dropoutDense):
        for numNode in layerDense:
            x = Dense(numNode)(x)
            if batchNorm :
                x = BatchNormalization(epsilon=1e-5)(x)
            if actDense == 'leaky_relu':
                x = LeakyReLU()(x)
            else:
                x = Activation(actDense)(x)
            if dropoutDense > 0:
                x = Dropout(0.5)(x)
        return x

    
    def _encoding(self, layerDense, batchNorm, actDense, dropoutDense):
        dimEncode = self._dimEncode
        x = self._inputs

        # Stack of Dense layers
        x = self._stackDense(x, layerDense[:-1], batchNorm, actDense, dropoutDense)
        
        # Build the mean and variance layers
        self._zMean = Dense(self._dimEncode)(x)
        self._zSigmaLog = Dense(self._dimEncode)(x) # log for linear dense

        # Define the sampling function for the sampling layer.
        # Note that the function must be in the same location with encoding for saving/loadind model.
        def sampling(args, stdEps):
            zMean, zSigmaLog = args
            epsilon = K.random_normal(shape=(K.shape(zMean)[0], K.shape(zMean)[1]),
                                      mean=0., stddev=stdEps)
            return zMean + K.exp(zSigmaLog) * epsilon  
        
        # Construct the latent as the output and build the encorder pipeline
        z = Lambda(sampling, arguments={'stdEps':self._stdEps})([self._zMean, self._zSigmaLog])
        self.encoder = Model(self._inputs, z, name='encoder')

        
    def _decoding(self, layerDense, batchNorm, actDense, dropoutDense):
         # Build the Decoder Model
        inputLatent = Input(shape=(self._dimEncode,), name='decoder_input')
        x = inputLatent
        
        # Stack of Dense layers
        x = self._stackDense(x, layerDense[-2::-1], batchNorm, actDense, dropoutDense)
            
        # Reconstruct the pixels as the output and build the decorder pipeline
        outputs = Dense(self._dimInput, activation='sigmoid', name='decoder_output')(x)
        self.decoder = Model(inputLatent, outputs, name='decoder')
        
        
    def _lossVAE(self, tensorInput, tensorDecode):
        zMean = self._zMean
        zSigmaLog = self._zSigmaLog
        ratRecon = self._ratRecon
        
        lossRecon =  binary_crossentropy(K.flatten(tensorInput), K.flatten(tensorDecode))
#         lossRecon =  mean_squared_error(K.flatten(tensorInput), K.flatten(tensorDecode))
        lossKL = - 0.5 * K.sum(1 + 2 * zSigmaLog - K.square(zMean) - K.square(K.exp(zSigmaLog)), axis=-1)
        return ratRecon * lossRecon + (1 - ratRecon) * lossKL
        
        
    def fit(self,
            xTrain, xValid,
            numEpochs=50, sizeBatch=32, nameOptim='adam', metrics=None,
            pathTempBest=None, patience=3,
           ):
        """
        The method is for training process. 
        The users can call this method easily just putting training and validation datasets.
        The dimension of dataset is determined by [#instance, *dimInput].
        For example, dimInput is flatten as a number and the dimension of dataset is [#instance, #feature]
        If dimInput is a list to represent [width, height, channels], the dimension of dataset is [#instance, width, height, channels]
        Args:
            xTrain (numpy ndarray): the training dataset.
            xValid (numpy ndarray): the validation dataset.
            numEpochs (int): the maximal epochs for training. Default is 50.
            sizeBatch (int): the batch size. Default is 32.
            nameOptim (string): the method for optimization. Default is adam.
            metrics (list(string or keras metrics)): the usage is the same with keras metrics for compile 
            pathTempBest (string): the temperory path of the best model for early-stop. Default None means without early-stop. 
            patience (int): the times of epochs to allow further trying if current loss is not better than the best. 
        Returns:
            history (keras.callbacks.History): the learning curving for the training process
            timeTrain (float): the consuming time of the training 
        """
        self.autoencoder.compile(optimizer=nameOptim, loss=self._lossVAE, metrics=metrics)

        if pathTempBest is None:
            callbacks = None
        else:
            if not os.path.exists(pathTempBest): # make sure the folder exists
                os.makedirs(pathTempBest)
            
            name_temp = "AutoEncoder" + str(time()) # use timestamp as unique name
            cbEarlyStop = EarlyStopping(monitor='val_loss', patience=patience, verbose=1, mode='auto')
            chkpt = pathTempBest + "/" + name_temp + ".hdf5"
            cbCheckPoint = ModelCheckpoint(filepath = chkpt, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
            callbacks = [cbEarlyStop, cbCheckPoint]
        
        # Train the autoencoder
        tic = time()
        history = self.autoencoder.fit(xTrain, xTrain,
                                       epochs=numEpochs,
                                       batch_size=sizeBatch, shuffle=True,
                                       callbacks=callbacks,
                                       validation_data=(xValid, xValid)
                                      )
        timeTrain = time() - tic
        
        # Assure the models are resuming from the best models
        if pathTempBest is not None:
            self.autoencoder = keras.models.load_model(chkpt, custom_objects={"_lossVAE": self._lossVAE})
            self.encoder = self.autoencoder.layers[1]
            self.decoder = self.autoencoder.layers[2]
        
        return history, timeTrain
    
    def save(self, pathFolder):
        """
        Deprecated! Because the availability for saving/loading model is limited.
        The method is to save the models of autoencoders as several hdf5 files in a given path of the folder.
        Args:
            pathFile (string): the given path of the folder where contains encoder, decoder, and autoencoder
        Returns:
            msg (string): the message for the saving process
        """
        # Create the message for saving model
        msg = ''
        if os.path.exists(pathFolder):
            msg += 'There is a existing folder.\r\n'
        else:
            os.makedirs(pathFolder)
            msg += 'Create a new folder.\r\n'
        
        # Save the models of encoder and decoder with save()
        self.encoder.save(pathFolder+'/encoder.h5')
        self.decoder.save(pathFolder+'/decoder.h5')
        
        # Save the model of autoencoder with json and save_weights(). 
        # Because autoencoder contains special loss function and sample function.
        self.autoencoder.save_weights(pathFolder+'/weightAutoencoder.h5')
        with open(pathFolder+'/configAutoencoder.json', 'w') as jsonFile:
            jsonFile.write(self.autoencoder.to_json())
        msg += 'successful.\r\n'
        return msg
    
    @staticmethod
    def load(pathFolder):
        """
        The method is to save the models of autoencoders as several hdf5 files in a given path of the folder.
        Args:
            pathFile (string): the given path of the folder where contains encoder, decoder, and autoencoder
        Returns:
            encoder (keras model): the model of encoder
            decoder (keras model): the model of decoder
            autoencoder (keras model) : the model of autoencoder. autoencoder.predict(x) is equivalent to decoder.predict(encoder.predict(x))
        """
        encoder = keras.models.load_model(pathFolder+'/encoder.h5')
        decoder = keras.models.load_model(pathFolder+'/decoder.h5')
        
        with open(pathFolder+'/configAutoencoder.json', 'r') as jsonFile:                              
            jsonConfig = jsonFile.readlines()[0]
        autoencoder = model_from_json(jsonConfig)
        autoencoder.load_weights(pathFolder+'/weightAutoencoder.h5')
        return encoder, decoder, autoencoder

# Convolutinal VAE

In [12]:
class ConvVAE(VAE):
    
    def __init__(self, 
                 dimInput, 
                 layerConv=[8, 32], sizeKernel=3, strides=2, actConv='leaky_relu', padding='same',
                 layerDense=[64, 2], actDense='leaky_relu', dropoutDense=0.5,
                 batchNorm = True,
                 ratRecon=0.998):
        """
        The basic properties and pipeline will be defined in the initialization.
        The dimension of input should be a form of a picture presented by a list [width, height, channels].
        It should be noted that layerDense defines the first half(encoder) of network. 
        The decoder will be reflected structure.
        For example, [64, 16, 2] means the nodes of decoder will be [2, 16, 64].
        It is similar for layerConv but decoder is not purely symmetric for convolution layers for this version.
        There is another parameter should noted that ratRecon=0.5 doesn't mean the effect is half.
        Because KL loss and reconstruction loss are not the same scale.
        Args:
            dimInput (list[int]): the dimension of input. E.g. [32, 28, 3] means 32 by 28 RGB pixels.
            layerConv (list[int]): the numbers of each convolution layer. Default is [8, 32].
            sizeKernel (int): the size of filter kernel. Default 3 means 3 by 3.
            strides (int): the stride for convolution. Default is 2.
            actConv (string): the activation function of each convolution layer. Default is 'leaky_relu'.
            padding (string): the padding method for convolution. Default is 'same'.
            layerDense (list[int]): the numbers of each dense layer. Default is [64, 2].
            actDense (string): the activation function of each dense layer. Default is 'leaky_relu'.
            dropoutDense (float): the dropout layer. Default is 0.5.
            ratRecon (float): the parameter for tuning the effects between KL loss and reconstruction loss.
        """
        
        # Initialize some setting 
        self._dimInput = dimInput # dimInput is (width, height, channels)
        self._inputs = Input(shape=(dimInput)) 
        self._dimEncode = layerDense[-1]
        self._ratRecon = ratRecon
        
        self._encoding(layerConv, sizeKernel, strides, actConv, padding,
                      layerDense, batchNorm, actDense, dropoutDense
                     )
        
        self._decoding(layerConv, sizeKernel, strides, actConv, padding,
                      layerDense, batchNorm, actDense, dropoutDense
                     )
        
        self.autoencoder = Model(self._inputs, self.decoder(self.encoder(self._inputs)), name='autoencoder')
    
    
    def _stackConv(self, x, 
                   layerConv, sizeKernel, strides, padding,
                   batchNorm, actConv, isTrans=False):
        for filters in layerConv:
            if isTrans:
                x = Conv2DTranspose(filters=filters,
                                    kernel_size=sizeKernel,
                                    strides=strides,
                                    padding=padding)(x)
            else:
                x = Conv2D(filters=filters,
                           kernel_size=sizeKernel,
                           strides=strides,
                           padding=padding)(x)
            if batchNorm :
                x = BatchNormalization(epsilon=1e-5)(x)
            if actConv == 'leaky_relu':
                x = LeakyReLU()(x)
            else:
                x = Activation(actConv)(x)
        return x

        
    def _encoding(self,
                 layerConv, sizeKernel, strides, actConv, padding,
                 layerDense, batchNorm, actDense, dropoutDense
                ):
        dimEncode = self._dimEncode
        x = self._inputs
        
        # Stack of Conv2D layers
        x = self._stackConv(x, 
                            layerConv, sizeKernel, strides, padding,
                            batchNorm, actConv)

        # Shape info needed to build Decoder Model
        self._shapeLastConv = K.int_shape(x)

        # Stack of Dense layers
        x = Flatten()(x)
        x = self._stackDense(x, layerDense[:-1], batchNorm, actDense, dropoutDense)
        
        # Build the mean and variance layers
        self._zMean = Dense(dimEncode)(x)
        self._zSigmaLog = Dense(dimEncode)(x) # log for linear dense

        # Define the sampling function for the sampling layer.
        # Note that the function must be in the same location with encoding for saving/loadind model.
        def sampling(args, stdEps):
            zMean, zSigmaLog = args
            epsilon = K.random_normal(shape=(K.shape(zMean)[0], K.shape(zMean)[1]),
                                      mean=0., stddev=stdEps)
            return zMean + K.exp(zSigmaLog) * epsilon  
        
        # Construct the latent as the output and build the encorder pipeline
        z = Lambda(sampling, arguments={'stdEps':self._stdEps})([self._zMean, self._zSigmaLog])
        self.encoder = Model(self._inputs, z, name='encoder')

        
    def _decoding(self,
                 layerConv, sizeKernel, strides, actConv, padding,
                 layerDense, batchNorm, actDense, dropoutDense
                ):
        
        shapeLastConv = self._shapeLastConv
         # Build the Decoder Model
        inputLatent = Input(shape=(self._dimEncode,), name='decoder_input')
        x = inputLatent
        
        # Stack of Dense layers
        x = self._stackDense(x, layerDense[-2::-1], batchNorm, actDense, dropoutDense)
        x = Dense(shapeLastConv[1] * shapeLastConv[2] * shapeLastConv[3])(x)
        x = Reshape((shapeLastConv[1], shapeLastConv[2], shapeLastConv[3]))(x)

        # Stack of Transposed Conv2D layers
        x = self._stackConv(x, 
                            layerConv[::-1], sizeKernel, strides, padding,
                            batchNorm, actConv, isTrans=True)

        # Build the Conv2DTranspose layer for the pixel dimension
        x = Conv2DTranspose(filters=self._dimInput[-1],
                            kernel_size=sizeKernel,
#                             strides=strides,
                            padding=padding)(x)

        # Reconstruct the pixels as the output and build the decorder pipeline
        outputs = Activation('sigmoid', name='decoder_output')(x)
        self.decoder = Model(inputLatent, outputs, name='decoder')

# For fast testing

In [7]:
# from time import time
# import unittest
# import numpy as np
# import pickle
# import keras
# from sklearn.metrics import silhouette_score

# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = '1' 

# from keras.datasets import mnist
# import tensorflow as tf
# from keras.backend.tensorflow_backend import set_session
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# # config.gpu_options.per_process_gpu_memory_fraction = 0.48
# set_session(tf.Session(config=config))
# from keras.datasets import mnist
# import numpy as np
# (xTrain, yTrain), (xTest, yTest) = mnist.load_data()
# xTrain = xTrain.astype('float32') / 255.
# xTest = xTest.astype('float32') / 255.
# numTrain = len(xTrain)
# numTest = len(xTest)
# sizeDigit = xTrain.shape[1:]

# dimInput = np.prod(xTrain.shape[1:])
# xTrain = xTrain.reshape((numTrain, dimInput))
# xTest = xTest.reshape((numTest, dimInput))
# print(xTrain.shape)
# print(xTest.shape)

# numEpochs = 200
# sizeBatch = 1024
# sizeKernel = 3
# layerDense = [256, 128, 64, 32, 2]
# ratRecon = 1
# nameOptim = 'adam'
# pathTempBest = '../model/temp/'
# patience = 2
# stdEps = 1.0

# vae = VAE(dimInput, layerDense=layerDense, ratRecon=ratRecon)
# history, timeTrain = vae.fit(xTrain, xTest, 
#                              numEpochs=numEpochs,
#                              sizeBatch=sizeBatch,
#                              pathTempBest=pathTempBest)

(60000, 784)
(10000, 784)
256
128
64
32
32
64
128
256
Train on 60000 samples, validate on 10000 samples
Epoch 1/200
60000/60000 [==============================] - 6s 97us/step - loss: 0.4493 - val_loss: 0.3151

Epoch 00001: val_loss improved from inf to 0.31514, saving model to ../model/temp//AutoEncoder1536569001.4604788.hdf5
Epoch 2/200
60000/60000 [==============================] - 2s 27us/step - loss: 0.2813 - val_loss: 0.2712

Epoch 00002: val_loss improved from 0.31514 to 0.27119, saving model to ../model/temp//AutoEncoder1536569001.4604788.hdf5
Epoch 3/200
60000/60000 [==============================] - 2s 27us/step - loss: 0.2626 - val_loss: 0.2557

Epoch 00003: val_loss improved from 0.27119 to 0.25565, saving model to ../model/temp//AutoEncoder1536569001.4604788.hdf5
Epoch 4/200
60000/60000 [==============================] - 2s 29us/step - loss: 0.2542 - val_loss: 0.2491

Epoch 00004: val_loss improved from 0.25565 to 0.24909, saving model to ../model/temp//AutoEncoder15365690

60000/60000 [==============================] - 1s 24us/step - loss: 0.2354 - val_loss: 0.2278

Epoch 00036: val_loss did not improve from 0.22774
Epoch 37/200
60000/60000 [==============================] - 1s 24us/step - loss: 0.2354 - val_loss: 0.2276

Epoch 00037: val_loss improved from 0.22774 to 0.22762, saving model to ../model/temp//AutoEncoder1536569001.4604788.hdf5
Epoch 38/200
60000/60000 [==============================] - 1s 23us/step - loss: 0.2353 - val_loss: 0.2276

Epoch 00038: val_loss improved from 0.22762 to 0.22762, saving model to ../model/temp//AutoEncoder1536569001.4604788.hdf5
Epoch 39/200
60000/60000 [==============================] - 1s 24us/step - loss: 0.2351 - val_loss: 0.2272

Epoch 00039: val_loss improved from 0.22762 to 0.22720, saving model to ../model/temp//AutoEncoder1536569001.4604788.hdf5
Epoch 40/200
60000/60000 [==============================] - 1s 24us/step - loss: 0.2350 - val_loss: 0.2273

Epoch 00040: val_loss did not improve from 0.22720
Epoch


Epoch 00074: val_loss improved from 0.22220 to 0.21966, saving model to ../model/temp//AutoEncoder1536569001.4604788.hdf5
Epoch 75/200
60000/60000 [==============================] - 1s 24us/step - loss: 0.2276 - val_loss: 0.2175

Epoch 00075: val_loss improved from 0.21966 to 0.21746, saving model to ../model/temp//AutoEncoder1536569001.4604788.hdf5
Epoch 76/200
60000/60000 [==============================] - 2s 26us/step - loss: 0.2263 - val_loss: 0.2159

Epoch 00076: val_loss improved from 0.21746 to 0.21592, saving model to ../model/temp//AutoEncoder1536569001.4604788.hdf5
Epoch 77/200
60000/60000 [==============================] - 2s 26us/step - loss: 0.2255 - val_loss: 0.2149

Epoch 00077: val_loss improved from 0.21592 to 0.21494, saving model to ../model/temp//AutoEncoder1536569001.4604788.hdf5
Epoch 78/200
60000/60000 [==============================] - 2s 25us/step - loss: 0.2248 - val_loss: 0.2141

Epoch 00078: val_loss improved from 0.21494 to 0.21407, saving model to ../mode

Epoch 112/200
60000/60000 [==============================] - 2s 26us/step - loss: 0.2190 - val_loss: 0.2075

Epoch 00112: val_loss did not improve from 0.20742
Epoch 113/200
60000/60000 [==============================] - 2s 27us/step - loss: 0.2189 - val_loss: 0.2074

Epoch 00113: val_loss did not improve from 0.20742
Epoch 114/200
60000/60000 [==============================] - 2s 29us/step - loss: 0.2190 - val_loss: 0.2074

Epoch 00114: val_loss did not improve from 0.20742
Epoch 00114: early stopping
